In [1]:
%matplotlib inline

import cv2
import itertools as it
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
import random
from utilities import *

In [2]:
images = list(pickle_iter(r"D:\Valorant\Killed\Color\00001.pickle"))
len(images)

8000

In [3]:
for i, image in enumerate(images):
    if show_and_wait(image, wait_time=1) == 'q':
        print(i)
        break
cv2.destroyAllWindows()

In [4]:
def fn(image):
    return cv2.resize(image, (30, 30))
g = grouper(images[6096:6296], 25)
g = (np.hstack(list(map(fn, g))) for g in g)
show_and_wait(np.vstack(list(g)))
cv2.destroyAllWindows()

In [5]:
positives = images[6096:6296]
list(prime_factors(6296 - 6096))

[2, 2, 2, 5, 5]

In [6]:
for i, image in enumerate(positives):
    if show_and_wait(image, wait_time=25) == 'q':
        print(i)
        break
cv2.destroyAllWindows()

In [7]:
top, left, bottom, right = 58, 1, 70, 59
show_and_wait(positives[0][top:bottom, left:right])
cv2.destroyAllWindows()

In [8]:
def fn(images):
    images = np.stack([i[top:bottom, left:right] for i in images])
    color = images.mean(axis=(0, 1, 2))
    show_and_wait(np.broadcast_to(color, (99, 99, 3)))
    cv2.destroyAllWindows()
    return color
color = fn(positives)
color

array([0.6616154, 0.7753435, 0.799254 ], dtype=float32)

In [9]:
template = np.broadcast_to(color, (bottom - top, right - left, 3))
match = cv2.matchTemplate(images[6100][:, left:right], template, cv2.TM_SQDIFF)
show_and_wait(np.broadcast_to(match / match.max(), (match.shape[0], 99)))
cv2.destroyAllWindows()
match.reshape(-1), match.min(), len([v for v in match.reshape(-1) if v < 20])

(array([154.25468 , 153.04462 , 151.79317 , 150.5627  , 149.67244 ,
        149.11552 , 148.67863 , 148.27658 , 147.96683 , 147.6722  ,
        147.59773 , 147.65987 , 147.82755 , 148.15913 , 148.37805 ,
        148.44377 , 148.34521 , 148.16124 , 147.90617 , 147.5979  ,
        147.57054 , 147.76909 , 148.09889 , 148.37741 , 148.79408 ,
        149.13243 , 149.57878 , 150.2237  , 151.11913 , 152.00108 ,
        153.14874 , 154.24525 , 154.89397 , 155.38318 , 155.89937 ,
        156.47836 , 157.01256 , 157.61873 , 158.15782 , 158.65344 ,
        159.1947  , 160.14236 , 161.01213 , 162.15965 , 162.84917 ,
        161.7648  , 154.80487 , 142.23798 , 129.57593 , 116.92341 ,
        104.39385 ,  91.93263 ,  79.39941 ,  66.61106 ,  53.783222,
         40.740993,  28.336264,  17.476507,  12.227976,  11.809668,
         19.994505,  49.439297,  78.6732  , 108.28246 , 137.36446 ,
        164.78253 , 191.68756 , 221.02502 , 249.36786 , 276.10706 ,
        313.89178 , 347.38953 , 372.23566 , 373.

In [10]:
def fn(image):
    match = cv2.matchTemplate(image[:, left:right], template, cv2.TM_SQDIFF)
    return match
image = np.hstack(list(map(fn, positives)))
show_and_wait(image / image.max())
cv2.destroyAllWindows()

In [11]:
negatives = [images[i] for i in sorted(random.sample(range(6000), 200))]
g = (np.hstack([cv2.resize(image, (55, 55)) for image in g]) for g in grouper(negatives, 20))
show_and_wait(np.vstack(list(g)))
cv2.destroyAllWindows()

In [12]:
len(positives), positives[0].shape, len(negatives), negatives[0].shape

(200, (108, 168, 3), 200, (108, 168, 3))

In [13]:
def fn(image):
    n, d = 40, math.log(2)
    match = cv2.matchTemplate(cv2.cvtColor(image[:, left:right], cv2.COLOR_BGR2LUV), cv2.cvtColor(template, cv2.COLOR_BGR2LUV), cv2.TM_SQDIFF)
    print(match.min(), match.max(), round(math.log(match.min(), 2)), round(math.log(match.max(), 2)), len([v for v in match.reshape(-1) if v < 2*match.min()]))
    match = np.clip(match, 0, 2**18)
    match = np.broadcast_to(match / match.max(), (match.shape[0], n))
    return np.hstack([cv2.cvtColor(cv2.resize(match, (n, n)), cv2.COLOR_GRAY2BGR), cv2.resize(image, (n, n))])
g = grouper(it.chain(negatives, 20 * [np.zeros_like(negatives[0])], positives), 20)
g = (np.hstack(list(map(fn, g))) for g in g)
show_and_wait(np.vstack(list(g)))
cv2.destroyAllWindows()

798614.75 2207800.5 20 21 38
1460958.2 2207800.5 20 21 97
801617.25 2154340.5 20 21 37
816758.5 2164517.2 20 21 39
817315.75 2155829.5 20 21 38
747795.75 2157465.5 20 21 32
805512.2 2166832.5 20 21 35
1517262.4 2164257.2 21 21 97
1519609.0 2165351.8 21 21 97
1512834.4 2189835.5 21 21 97
1471356.4 2195144.5 20 21 97
1449167.8 2147328.2 20 21 97
1461985.2 2167595.0 20 21 97
1457824.1 2172552.0 20 21 97
1456519.5 2172534.5 20 21 97
1424924.2 2154240.0 20 21 97
1453344.4 2153865.0 20 21 97
1453315.1 2164013.0 20 21 97
1443195.1 2173254.8 20 21 97
1454624.0 2175756.8 20 21 97
1458385.1 2164222.0 20 21 97
1458385.1 2164222.0 20 21 97
1458886.0 2164222.0 20 21 97
1433619.1 2172651.5 20 21 97
1310930.1 2165713.5 20 21 97
1367151.8 2178283.0 20 21 97
1370257.2 2180537.2 20 21 97
1370483.4 2164981.5 20 21 97
1513160.2 2167761.5 21 21 97
1519972.8 2165484.8 21 21 97
1521227.1 2165148.5 21 21 97
1491787.2 2166769.8 21 21 97
1491787.2 2166769.8 21 21 97
1492256.5 2182043.2 21 21 97
1508605.4 216242

24802.922 2309530.0 15 21 5
27966.838 2309735.2 15 21 5
27966.838 2309645.2 15 21 5
28126.615 2306436.2 15 21 5
28126.615 2306436.2 15 21 5
29312.697 2311295.2 15 21 5
29312.697 2311295.2 15 21 5
29312.697 2311295.2 15 21 5
29312.697 2311159.0 15 21 5
29312.697 2311159.0 15 21 5
29312.697 2311159.0 15 21 5
29312.697 2311203.5 15 21 5
29312.697 2311143.5 15 21 5
29312.697 2312994.5 15 21 5
29312.697 2313464.0 15 21 5
28009.65 2313508.8 15 21 5
28009.65 2313508.8 15 21 5
28009.65 2318968.0 15 21 5
28056.162 2318968.0 15 21 5
28056.162 2319166.8 15 21 5
28056.162 2319266.0 15 21 5
28124.676 2319222.2 15 21 5
28124.676 2319222.2 15 21 5
28124.676 2319222.2 15 21 5
28124.676 2319222.2 15 21 5
28124.676 2310033.0 15 21 5
28124.676 2308625.5 15 21 5
28124.676 2308625.5 15 21 5
28124.676 2308625.5 15 21 5
28124.676 2309373.2 15 21 5
28124.676 2309450.5 15 21 5
28124.676 2309450.5 15 21 5
28006.42 2309450.5 15 21 5
28148.787 2309450.5 15 21 5
28148.787 2309450.5 15 21 5
28148.787 2309450.5 15 2

In [14]:
def fn(image):
    n, d = 40, math.log(2)
    match = cv2.matchTemplate(cv2.cvtColor(image[:, left:right], cv2.COLOR_BGR2LUV), cv2.cvtColor(template, cv2.COLOR_BGR2LUV), cv2.TM_SQDIFF)
    if image.max() == image.min() == 0:
        print()
    else:
        print(match.min() < 500000 and len([v for v in match.reshape(-1) if v < 2*match.min()]) < 30)
    match = np.clip(match, 0, 2**18)
    match = np.broadcast_to(match / match.max(), (match.shape[0], n))
    return np.hstack([cv2.cvtColor(cv2.resize(match, (n, n)), cv2.COLOR_GRAY2BGR), cv2.resize(image, (n, n))])
g = grouper(it.chain(negatives, 20 * [np.zeros_like(negatives[0])], positives), 20)
g = (np.hstack(list(map(fn, g))) for g in g)
show_and_wait(np.vstack(list(g)))
cv2.destroyAllWindows()

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals